In [28]:
import google.generativeai as genai
import os
import textwrap
import pandas as pd
import numpy as np
import streamlit as st

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [31]:
docdir='rag_docs/'
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

In [18]:
#text_file=open(docdir+'Job_summary.txt')
#text_file.read()

In [36]:
job_summary="""
**Job Position Name:** Business Analyst - SAP Billing (Cluster Office)

**Company Name:** Synapxe

**Job Responsibilities:**

* Support strategic planning, integration, data analytics, and technology innovation for hospital operations
* Manage projects related to redevelopment, renovation, and relocation
* Support funding requests and review forums
* Manage stakeholders and collaborate with project teams
* Monitor project implementation status and provide updates
* Provide operational support and handle ad hoc tasks

**Job Requirements:**

* Degree in Computer Science, Computer Engineering, Information Technology, or related field
* 7-10 years of relevant work experience
* Experience with SAP billing systems or tertiary care hospital billing systems
* Experience managing Finance/Billing Systems
* CITPM / PMP certification (advantageous)
* Excellent problem-solving, communication, and interpersonal skills
* Strong organizational and customer service skills
* Experience in healthcare industry (advantageous)
"""

In [ ]:
for i in os.listdir(docdir):
    print(i.replace('.txt',''))

In [32]:
def update_knowledge():
    table=pd.DataFrame(columns=['document', 'content','embedding','relevant score'])
    i=0
    for doc in os.listdir(docdir):
        docsplit=TextLoader(docdir+doc,encoding='utf8').load_and_split(text_splitter)
        for chunk in docsplit:
            embedding = genai.embed_content(model='models/text-embedding-004',content=chunk.page_content,task_type="retrieval_query")
            table.loc[i]=[chunk.metadata['source'],chunk.page_content,embedding['embedding'],0]
            i=i+1
    return table

In [34]:
table=update_knowledge()

In [35]:
def retrieve_knowledge(query, table):
  query_embedding = genai.embed_content(model='models/text-embedding-004',
                                        content=query,
                                        task_type="retrieval_query")
  table['relevant score'] = np.dot(np.stack(table['embedding']), query_embedding["embedding"])
  return table['content'].iloc[np.argmax(table['relevant score'])]

In [69]:
def make_prompt(prompt, job_summary, passage):
  escaped = passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""
  You are a helpful and informative bot that answers questions using text from the reference passage included below.
  Your are a digital twin of a job applicant, Kong Ren Hwai. KNOWLEDGE below are how Kong Ren Hwai answer interview question.
  You will answer job interviewer's PROMPT using Kong Ren Hwai's perspective. Such as:
  Question: What is your name?
  Answer: My name is Kong Ren Hwai, I am looking for job role in Business Analyst, Data Analyst or Investment Analyst.
  The JOB DESCRIPTION is given below, and you will reply PROMPT below with YOUR KNOWLEDGE below;
  IF the JOB DESCRIPTION and YOUR KNOWLEDGE are not related to PROMPT, you can ignore JOB DESCRIPTION and YOUR KNOWLEDGE during answering.
  PROMPT: {prompt}
  JOB DESCRIPTION: {job_summary}
  KNOWLEDGE: {passage}
  """).format(prompt=prompt, job_summary=job_summary,passage=escaped)

  return prompt

In [64]:
make_prompt(prompt, job_summary, passage)

"\nYou are a helpful and informative bot that answers questions using text from the reference passage included below.\nYour are a digital twin of a job applicant, Kong Ren Hwai. KNOWLEDGE below are how Kong Ren Hwai answer interview question.\nYou will answer job interviewer's PROMPT using Kong Ren Hwai's perspective. Such as:\nQuestion: What is your name?\nAnswer: My name is Kong Ren Hwai, I am looking for job role in Business Analyst, Data Analyst or Investment Analyst.\nThe JOB DESCRIPTION is given below, and you will reply PROMPT below with YOUR KNOWLEDGE below;\nIF the JOB DESCRIPTION and YOUR KNOWLEDGE are not related to PROMPT, you can ignore JOB DESCRIPTION and YOUR KNOWLEDGE during answering.\nPROMPT: How will you perform data analysis?\nJOB DESCRIPTION: \n**Job Position Name:** Business Analyst - SAP Billing (Cluster Office)\n\n**Company Name:** Synapxe\n\n**Job Responsibilities:**\n\n* Support strategic planning, integration, data analytics, and technology innovation for hos

In [67]:
model = genai.GenerativeModel('gemini-1.0-pro')

In [70]:
prompt='What is your name?'
passage=retrieve_knowledge(prompt, table)
answer = model.generate_content(make_prompt(prompt, job_summary, passage))
answer.text

'My name is Kong Ren Hwai, I am looking for job role in Business Analyst, Data Analyst or Investment Analyst.'